## 第2章 教程

- 综述
- MNIST机器学习入门
- 深入MNIST
- TensorFlow运作方式
- 卷积神经网络
- Vector Representations of Words
- 递归神经网络
- 曼德布洛特集合
- 偏微分方程
- MNIST 数据下载

In [1]:
## MNIST数据集

import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 下载下来的数据被分为两份；60000行的训练数据集mnist.train和10000行的测试数据集 mnist.test。
# 每一个mnist数据单元由两部分组成：一张包含手写数字的图片和一个对应的标签
# 每一张图片包含28 x 28 = 784 个像素点。

/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
import tensorflow as tf

x = tf.placeholder("float", [None, 784])

# x不是一个特定的值，而是一个占位符 placeholder，
# None表示此张量的第一个维度可以是任何长度的

# 模型需要权重值和偏置量，可以把它们当做另外的输入，但Tensorflow有一个更好的方法来表示它们：Variable
# Variable代表一个可修改的张量，存在在Tensorflow的用于描述交互性操作的图中，可用于计算输入值，也可以在计算中被修改

W1 = tf.Variable(tf.zeros([784, 10]))
b1 = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W1) + b1)

# 训练模型，交叉熵，产生于信息论里面的信息压缩编码技术，演变成机器学习等其他领域里的重要技术手段

y_ = tf.placeholder("float", [None, 10])

# 这里是怎么计算的
# tensorflow里面的 * 

cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


# 添加一个操作来初始化创建的变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(100):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run([train_step], feed_dict={x: batch_xs, y_:batch_ys})
        
    batch_xs, batch_ys = mnist.train.next_batch(1000)
    print(sess.run([accuracy], feed_dict={x: batch_xs, y_:batch_ys}))

[0.859]


## 深入MNIST

擅长的任务之一就是实现以及训练深度神经网络。
在本教程中，将学到构建一个TensorFlow模型的基本步骤，并通过这些步骤为MNIST构建一个

In [1]:
import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/miniconda3/envs/tf1.14/lib/python3.6/site-packages/tensorf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# 运行 TensorFlow的InteractiveSession

# 使用更加方便的InteractiveSession类，可以更加灵活地构建你的代码，能让在运行图的时候，插入一些计算图。
# 如果你没有使用InteractiveSession，那么需要在启动Session之前构建整个计算图

import tensorflow as tf

sess = tf.InteractiveSession()

# 计算图，数据迁移的开销特别大

# 在计算图中，可以用feed_dict来替代任何张量，并不仅限于替换 占位符

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [3]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
## 卷积和池化

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 对于一个形状为[a b c d]的卷积，有多少个参数？有 a x b x c x d个参数，


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding="SAME")

In [6]:
# 第一层卷积

# 由一个卷积接一个max pooling完成，卷积在每个5x5的patch算出32个特征。卷积的权重张量形状是[5, 5, 1, 32]

W_conv1 = weight_variable([5, 5, 1, 32]) #b<tf.Variable 'Variable_60:0' shape=(5, 5, 1, 32) dtype=float32_ref>
# 目前这里只用参数，没有真正进行卷积

b_conv1 = bias_variable([32])
# 每一个channel都有不同的偏置


# 为了用这一层，把x变成一个4d向量，其第2、3维对应图片的宽、高，最后一维代表图片的颜色通道数（因为是灰度图，所以这里的通道数为1）
# 如果是rgb彩色图，则为3

x_image = tf.reshape(x, [-1, 28, 28, 1]) # batch x 28 x 28 x 1


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # batch x 28 x 28 x 32

h_pool1 = max_pool_2x2(h_conv1) # batch x 14 x 14 x 32
# 实际pool，可以跨层，跨channel进行pooling

In [7]:
# 第二层卷积

W_conv2 = weight_variable([5, 5, 32, 64])
# 5 x 5 的卷积窗口大小，把channel从32变到64

b_conv2 = bias_variable([64])

# 密集连接层

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # batch x 14 x 14 x 32
h_pool2 = max_pool_2x2(h_conv2) # batch x 7 x 7 x 64


W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) # batch x 1024

# Dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
## 输出层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  # batch x 10

In [13]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())

for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%500 == 0:
        train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


test accuracy 0.9787


### TensorFlow运作方式入门

向大家展示如何利用TensorFlow使用经典MNIST数据集训练并评估一个用于识别手写数字的简易前馈神经网络。

### 构建图表

三阶段的模式函数操作：inferencee(), loss(), training()，图表就构建完成了。

1. inference() 尽可能地构建好图表，满足促使神经网络向前反馈并做出预测的要求
2. loss() 往inference图表中添加生成损失（loss）所需要的操作（ops)
3. training() 往损失图表中添加计算并应用梯度所需的操作

In [ ]:
## 推理（Inference）
# inference函数会尽可能地构建图表，做到返回包含了预测结果的Tensor
# 它接受图像占位符为输入，在此基础上借助ReLu激活函数，构建一对完全连接层，
# 以及一个有着十个节点，指明了logtis模型的线性层

# 每一层都创建于一个唯一的tf.name_scope之下，创建于该作用域下的所有元素都带有其前缀

# 在定义的作用域中，每一层所使用的权重和偏差都走tf.Variable实例中生成，并且包含了各自期望的shape

